In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
def read_data(folder):
    data = pd.DataFrame()
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            file_path = os.path.join(folder, file)
            df = pd.read_csv(file_path, header=None, names=['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z'], skiprows=1)
            df['activity'] = folder.split('/')[-1]
            data = pd.concat([data, df], ignore_index=True)
    return data


In [7]:
base_path = "/content/drive/My Drive/Colab Notebooks/"
# base_path = "./"
print(os.listdir(base_path))

['bestsellers with categories.csv', 'Hw1.ipynb', '2017_jun_final.csv', 'foo.txt', 'Hw2.2.ipynb', 'Hw2.3.ipynb', 'Hw2.1.ipynb', 'Housing.csv', 'Untitled (1)', 'Untitled1.ipynb', 'Копия блокнота "Hw3.ipynb" (1)', 'Копия блокнота "Hw3.ipynb"', 'Untitled0.ipynb', 'DS_HW3 (1).ipynb', 'DS_HW3.ipynb', 'Hw3.ipynb', 'bikes_rent.csv', 'DS_HW4.ipynb', 'Hw4.ipynb', 'homework.zip', 'data (1)', 'data', 'Untitled', 'Копия блокнота  Hw3 (1).ipynb', 'Untitled2.ipynb', 'Копия блокнота "Hw5.ipynb"', 'Hw5.ipynb']


In [8]:
idle_data = read_data(f'{base_path}data/idle')
idle_data.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.038307,6.148310,7.661446,idle
1,-0.062249,6.531382,7.110780,idle
2,-0.248997,6.268020,7.058107,idle
3,-0.205901,6.215348,7.627927,idle
4,-0.330400,6.090849,7.230490,idle


In [9]:
running_data = read_data(f'{base_path}data/running')
stairs_data = read_data(f'{base_path}data/stairs')
walking_data = read_data(f'{base_path}data/walking')


In [10]:
all_data = pd.concat([running_data, walking_data, stairs_data, idle_data], ignore_index=True)

In [18]:
print(idle_data.shape)
print(running_data.shape)
print(stairs_data.shape)
print(walking_data.shape)
print(all_data.shape)

(62340, 4)
(102240, 4)
(4950, 4)
(55500, 4)
(225030, 4)


In [19]:
y = all_data['activity']
# y = all_data[:, 3]
y.head()

0    running
1    running
2    running
3    running
4    running
Name: activity, dtype: object

In [27]:
X = all_data[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
# X = all_data[:, :3]
X.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,3.787627,-16.208746,2.398990
1,14.709975,34.596214,19.043480
2,-1.350330,13.680469,1.345541
3,5.363012,-5.410896,-8.130708
4,-2.820370,7.690176,4.242526


За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням.

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статті.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

In [21]:
from sklearn import datasets, svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LogisticRegression

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)

(157521, 3) (67509, 3)


#One Vs Rest Classifier

In [31]:
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’

clf_ovr_lin = SVC(kernel='linear', decision_function_shape='ovr', max_iter=100000).fit(X_train, y_train)
clf_ovr_lin.score(X_test, y_test)

y_pred_ovr_lin = clf_ovr_lin.predict(X_test)

y_t = y_test.tolist()
y_p = y_pred_ovr_lin.tolist()
n = 0
for i in range(y_test.shape[0]):
  if (y_t[i] != y_p[i]):
    n += 1

print(f'OneVsRestClassifier(SVC()): помилково = {n}, всього = {y_test.shape[0]}, точність = {(1 - n / y_test.shape[0])}' )

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


OneVsRestClassifier(SVC()): помилково = 47523, всього = 67509, точність = 0.2960494156334711


In [32]:
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’
clf_ovr_pol = SVC(kernel='poly', decision_function_shape='ovr', max_iter=100000).fit(X_train, y_train)
clf_ovr_pol.score(X_test, y_test)

y_pred_ovr_pol = clf_ovr_pol.predict(X_test)

y_t = y_test.tolist()
y_p = y_pred_ovr_pol.tolist()
n = 0
for i in range(y_test.shape[0]):
  if (y_t[i] != y_p[i]):
    n += 1

print(f'OneVsRestClassifier(SVC()): помилково = {n}, всього = {y_test.shape[0]}, точність = {(1 - n / y_test.shape[0])}' )

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


OneVsRestClassifier(SVC()): помилково = 18084, всього = 67509, точність = 0.7321246056081412


In [33]:
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’
clf_ovr_rbf = SVC(kernel='rbf', decision_function_shape='ovr', max_iter=100000).fit(X_train, y_train)
clf_ovr_rbf.score(X_test, y_test)

y_pred_ovr_rbf = clf_ovr_rbf.predict(X_test)

y_t = y_test.tolist()
y_p = y_pred_ovr_rbf.tolist()
n = 0
for i in range(y_test.shape[0]):
  if (y_t[i] != y_p[i]):
    n += 1

print(f'OneVsRestClassifier(SVC()): помилково = {n}, всього = {y_test.shape[0]}, точність = {(1 - n / y_test.shape[0])}' )

OneVsRestClassifier(SVC()): помилково = 6413, всього = 67509, точність = 0.9050052585581182


In [34]:
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’
clf_ovr_sig = SVC(kernel='sigmoid', decision_function_shape='ovr', max_iter=100000).fit(X_train, y_train)
ovr_sig_score = clf_ovr_sig.score(X_test, y_test)

y_pred_ovr_sig = clf_ovr_sig.predict(X_test)

y_t = y_test.tolist()
y_p = y_pred_ovr_sig.tolist()
n = 0
for i in range(y_test.shape[0]):
  if (y_t[i] != y_p[i]):
    n += 1

print(f'OneVsRestClassifier(SVC()): помилково = {n}, всього = {y_test.shape[0]}, точність = {(1 - n / y_test.shape[0])}, score = {ovr_sig_score}')

OneVsRestClassifier(SVC()): помилково = 25538, всього = 67509, точність = 0.6217096979661971, score = 0.6217096979661971


#One Vs One Classifier

In [35]:
clf_ovo = SVC(kernel='rbf', decision_function_shape='ovo', max_iter=100000, random_state=42).fit(X_train, y_train)
ovo_score = clf_ovo.score(X_test, y_test)

y_pred_ovo = clf_ovo.predict(X_test)

y_t = y_test.tolist()
y_p = y_pred_ovo.tolist()
n = 0
for i in range(y_test.shape[0]):
  if (y_t[i] != y_p[i]):
    n += 1

print(f'OneVsOneClassifier(SVC()): помилково = {n}, всього = {y_test.shape[0]}, точність = {(1 - n / y_test.shape[0])}, score = {ovo_score}')

OneVsOneClassifier(SVC()): помилково = 6413, всього = 67509, точність = 0.9050052585581182, score = 0.9050052585581182


In [30]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(max_depth=1000)
clf_rf = clf_rf.fit(X_train, y_train)
rf_score = clf_rf.score(X_test, y_test)
y_pred_rf = clf_rf.predict(X_test)

y_t = y_test.tolist()
y_p = y_pred_rf.tolist()
n = 0
for i in range(y_test.shape[0]):
  if (y_t[i] != y_p[i]):
    n += 1

print(f'RandomForestClassifier(): помилково = {n}, всього = {y_test.shape[0]}, точність = {(1 - n / y_test.shape[0])}, score = {rf_score}')

RandomForestClassifier(): помилково = 20, всього = 67509, точність = 0.9997037432046098, score = 0.9997037432046098


# One Vs Rest Classifier
### linear - Не досягає мети за прийнятний час.
### poly - Не досягає мети за прийнятний час.
### rbf - Досягає мети за прийнятний час, точність трохи більша за 90%.
### sigmoid - Невисока точність, 62%.
#-----------------------------------
# One Vs One Classifier
### rbf - Досягає мети за прийнятний час, точність трохи більша за 90%.
#### Інші функції не перевірялися.
#-----------------------------------
# Random Forest Classifier
## Швидкість та точність найвищи, точність майже 100%.
